https://github.com/jukedeck/nottingham-dataset/tree/master/MIDI/chords

Chord -> melody training

In [12]:
# Probably more imports than are really necessary...
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm
import librosa
import numpy as np
import miditoolkit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
import random

In [13]:
def get_midi_paths(root_dir):
    midi_paths = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith('.mid') or file.lower().endswith('.midi'):
                midi_paths.append(os.path.join(root, file))
    return sorted(midi_paths)

midi_root = 'nottingham-data/MIDI'
midi_files = get_midi_paths(midi_root)

In [14]:
def extract_chords_and_melody(midi_path):
    midi_obj = miditoolkit.midi.parser.MidiFile(midi_path)
    notes = [note for note in midi_obj.instruments[0].notes if not note.is_drum]
    
    # Sort notes
    notes.sort(key=lambda x: x.start)

    # Extract melody (assuming track 0 is melody)
    melody = []
    for note in notes:
        pitch = note.pitch
        start = note.start / midi_obj.ticks_per_beat
        duration = (note.end - note.start) / midi_obj.ticks_per_beat
        melody.append((pitch, start, duration))

    # Use built-in chord extraction (or a heuristic)
    chords = []  # Replace with a real extraction
    for chord in midi_obj.markers:
        chords.append((chord.text, chord.time / midi_obj.ticks_per_beat))

    return chords, melody


### Tokenize

In [15]:
def chord_to_token(chord_name):
    return f"[{chord_name}]"
def melody_to_tokens(melody, step=0.25):
    tokens = []
    for pitch, start, dur in melody:
        pitch_str = f"{pitch}_{round(dur, 2)}"
        tokens.append(pitch_str)
    return tokens


### Dataset Class

In [16]:
class ChordMelodyDataset(Dataset):
    def __init__(self, midi_dir):
        self.data = []
        for fname in os.listdir(midi_dir):
            if fname.endswith('.mid'):
                chords, melody = extract_chords_and_melody(os.path.join(midi_dir, fname))
                chord_tokens = [chord_to_token(c[0]) for c in chords]
                melody_tokens = melody_to_tokens(melody)
                if len(chord_tokens) > 0 and len(melody_tokens) > 0:
                    self.data.append((chord_tokens, melody_tokens))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        chords, melody = self.data[idx]
        return chords, melody


### Model Build

In [17]:
class ChordToMelody(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, chord_seq, melody_seq):
        embedded_chords = self.embedding(chord_seq)
        _, (hidden, cell) = self.encoder(embedded_chords)

        embedded_melody = self.embedding(melody_seq)
        output, _ = self.decoder(embedded_melody, (hidden, cell))
        logits = self.output(output)
        return logits


### Get Our Tokens

In [36]:


### --- STEP 1: Pair MIDI files from melody/ and chords/ ---
def get_melody_chord_pairs(midi_root):
    melody_dir = os.path.join(midi_root, 'melody')
    chords_dir = os.path.join(midi_root, 'chords')

    melody_files = sorted([f for f in os.listdir(melody_dir) if f.endswith('.mid')])
    chord_files = sorted([f for f in os.listdir(chords_dir) if f.endswith('.mid')])

    pairs = []
    for fname in melody_files:
        melody_path = os.path.join(melody_dir, fname)
        chord_path = os.path.join(chords_dir, fname)
        if os.path.exists(chord_path):
            pairs.append((chord_path, melody_path))
    return pairs

### --- STEP 2: Load and parse MIDI files ---
def load_pair(chord_path, melody_path):
    chords_midi = miditoolkit.MidiFile(chord_path)
    melody_midi = miditoolkit.MidiFile(melody_path)
    return chords_midi, melody_midi

### --- STEP 3: Extract melody tokens ---
def extract_melody_tokens(midi, quantize_step=0.25):
    notes = midi.instruments[0].notes
    ticks_per_beat = midi.ticks_per_beat
    tokens = []
    for note in notes:
        start_beat = note.start / ticks_per_beat
        duration_beat = round((note.end - note.start) / ticks_per_beat / quantize_step) * quantize_step
        pitch = note.pitch
        tokens.append(f"{pitch}_{duration_beat:.2f}")
    return tokens

### --- STEP 4: Extract chord tokens from markers ---
def extract_chord_tokens(midi):
    markers = midi.markers
    ticks_per_beat = midi.ticks_per_beat
    chord_tokens = []
    for marker in markers:
        beat = marker.time / ticks_per_beat
        chord_name = marker.text.strip()
        chord_tokens.append((beat, f"[{chord_name}]"))
    return chord_tokens

### --- STEP 5: Align chords to melody time grid ---
def align_chords(chord_tokens, melody_tokens, step=0.25):
    melody_len = len(melody_tokens)
    result = []
    current_chord = '[N]'  # default: no chord
    chord_idx = 0
    chord_times = [beat for beat, _ in chord_tokens]

    for i in range(melody_len):
        time = i * step
        while chord_idx + 1 < len(chord_tokens) and chord_times[chord_idx + 1] <= time:
            chord_idx += 1
        if chord_tokens:
            current_chord = chord_tokens[chord_idx][1]
        result.append(current_chord)
    return result

### --- STEP 6: Extract token pair for one file ---
def extract_token_pair(chord_path, melody_path, step=0.25):
    chords_midi, melody_midi = load_pair(chord_path, melody_path)
    melody_tokens = extract_melody_tokens(melody_midi, quantize_step=step)
    chord_tokens_raw = extract_chord_tokens(chords_midi)
    chord_tokens = align_chords(chord_tokens_raw, melody_tokens, step=step)
    return chord_tokens, melody_tokens

### --- STEP 7: Load all pairs into memory ---
def extract_all_token_pairs(midi_root, step=0.25):
    pairs = get_melody_chord_pairs(midi_root)
    all_data = []
    for chord_path, melody_path in pairs:
        try:
            chords, melody = extract_token_pair(chord_path, melody_path, step=step)
            if len(chords) == len(melody) and len(chords) > 0:
                all_data.append((chords, melody))
        except Exception as e:
            print(f"Failed on {chord_path} / {melody_path}: {e}")
    return all_data

### --- STEP 8: Example usage ---
midi_root = "nottingham-dataset/MIDI"
data = extract_all_token_pairs(midi_root, step=0.25)

print(f"✅ Extracted {len(data)} chord–melody token sequences.")
print("🎵 Sample chord tokens:", data[0][0][:10])
print("🎶 Sample melody tokens:", data[0][1][:10])


✅ Extracted 1021 chord–melody token sequences.
🎵 Sample chord tokens: ['[N]', '[N]', '[N]', '[N]', '[N]', '[N]', '[N]', '[N]', '[N]', '[N]']
🎶 Sample melody tokens: ['76_1.00', '74_2.00', '71_1.00', '69_1.50', '71_0.50', '72_1.00', '71_2.00', '67_1.00', '69_2.00', '76_1.00']


### Map

In [ ]:
# After collecting all tokenized data
token_set = set()
for chords, melody in dataset.data:
    token_set.update(chords)
    token_set.update(melody)

token2idx = {token: i for i, token in enumerate(sorted(token_set))}
idx2token = {i: token for token, i in token2idx.items()}

### Skeleton

In [ ]:
def collate_fn(batch):
    chord_batch, melody_batch = zip(*batch)
    # Convert to padded tensors here (you can use torch.nn.utils.rnn.pad_sequence)
    # Or just truncate/zero-pad manually for now
    return chord_tensor, melody_tensor

loader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

model = ChordToMelody(vocab_size=len(token2idx)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    for chords, melodies in loader:
        logits = model(chords, melodies[:, :-1])
        loss = loss_fn(logits.reshape(-1, logits.size(-1)), melodies[:, 1:].reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}: Loss {loss.item():.4f}")


In [35]:

def list_subdirectories(path):
    return [name for name in os.listdir(path)
            if os.path.isdir(os.path.join(path, name))]

midi_root = "nottingham-dataset/MIDI"
subdirs = list_subdirectories(midi_root)
print("📁 Subdirectories:", subdirs)
midi_root = "nottingham-dataset/MIDI"
print("✅ Exists:", os.path.exists(midi_root))
print("📁 Contents:", os.listdir(midi_root))

print("Current working directory:", os.getcwd())

📁 Subdirectories: ['chords', 'melody']
✅ Exists: True
📁 Contents: ['ashover1.mid', 'ashover10.mid', 'ashover11.mid', 'ashover12.mid', 'ashover13.mid', 'ashover14.mid', 'ashover15.mid', 'ashover16.mid', 'ashover17.mid', 'ashover18.mid', 'ashover19.mid', 'ashover2.mid', 'ashover20.mid', 'ashover21.mid', 'ashover22.mid', 'ashover23.mid', 'ashover24.mid', 'ashover25.mid', 'ashover26.mid', 'ashover27.mid', 'ashover28.mid', 'ashover29.mid', 'ashover3.mid', 'ashover30.mid', 'ashover31.mid', 'ashover32.mid', 'ashover33.mid', 'ashover34.mid', 'ashover35.mid', 'ashover36.mid', 'ashover37.mid', 'ashover38.mid', 'ashover39.mid', 'ashover4.mid', 'ashover40.mid', 'ashover41.mid', 'ashover42.mid', 'ashover43.mid', 'ashover44.mid', 'ashover45.mid', 'ashover46.mid', 'ashover5.mid', 'ashover6.mid', 'ashover7.mid', 'ashover8.mid', 'ashover9.mid', 'chords', 'hpps1.mid', 'hpps10.mid', 'hpps11.mid', 'hpps12.mid', 'hpps13.mid', 'hpps14.mid', 'hpps15.mid', 'hpps16.mid', 'hpps17.mid', 'hpps18.mid', 'hpps19.mid